## segment plotting with half decades, Segments4.ipynb

* 20240611 use distnace unit conversions form packages/convert/converters.py  (converters 0.1)
* .. from
 29.5.2024, start from segments.ipynb 21.5.2024 "added decade images"
* use half decade images
* https://chatgpt.com/share/9f0e536b-9277-4ba9-ac48-c5af49bd76e8

Function definitions first:

## and now setting images to side, and display scale in t user and engineering units...
* use functions convert_to_user_units and  ... _engineering units

In [1]:
# Segments.ipynb 11.6.2024

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.colors import NoNorm
import os
# import funcions from local package "convert"
from convert.converters import convert_to_engineering_units, convert_to_user_units  

# Directory containing images for half decades
image_dir = 'half_decades'

# Generate image paths with new naming convention and extended range
image_paths = {i / 2: os.path.join(image_dir, f'decade_{i / 2:.1f}.png') for i in range(-38, 56)}

# Mapping decade numbers to angles with the extended range and half decade increments
decade_to_angle = {i / 2: ((i / 2) * 6) % 360 for i in range(-38, 56)}

# segment plotting function
def plot_segment(ax, mid_angle, angle_width, mid_radius, radius_range, color='gray', alpha=0.4):
    start_angle = np.radians(mid_angle - angle_width / 2)
    end_angle = np.radians(mid_angle + angle_width / 2)
    angles_rad = np.linspace(start_angle, end_angle, 100)
    inner_radius = mid_radius - radius_range
    outer_radius = mid_radius + radius_range
    ax.fill_between(angles_rad, inner_radius, outer_radius, color=color, alpha=alpha, zorder=2)

# polar arrow cursor plotting update function
def update_polar_arrow(decade, min_radius, max_radius):
    decade_key = f'{decade:.1f}'
    angle = decade_to_angle[float(decade_key)]
    
    with output:
        clear_output(wait=True)
        fig = plt.figure(figsize=(12, 6))
        ax = fig.add_subplot(121, projection='polar')
        
        # Plot the image in a new axis on the right
        ax_inset = fig.add_axes([0.55, 0.1, 0.4, 0.8])  # Position and size of the image subplot
        image_path = image_paths[float(decade_key)]
        image = plt.imread(image_path)
        ax_inset.imshow(image)
        ax_inset.axis('off')  # Turn off axis for inset
        
        # Plotting segments and arrow on the polar plot
        num_segments = 12
        angle_width = 1 ; mid_radius = 1.8 ; radius_range = 0.2
        num_small_segments = 60
        small_segment_angle_width = 3; small_segment_mid_radius = 2.2; small_segment_radius_range = 0.05
        
        ax.spines['polar'].set_visible(False)
        
        for i in range(num_segments):
            mid_angle = i * (360 / num_segments)
            plot_segment(ax, mid_angle, angle_width, mid_radius, radius_range, 'green', 0.2)
        for i in range(num_small_segments):
            mid_angle = i * (360 / num_small_segments)
            plot_segment(ax, mid_angle, small_segment_angle_width, small_segment_mid_radius, small_segment_radius_range, 'lightblue', 0.5)
        
        plot_segment(ax,0,359,0.1,0.01,'gray', 0.2)  # plot center dot

        # plot red pointer line
        radians = np.deg2rad(angle)
        ax.plot([radians, radians], [min_radius, max_radius], color='red', linewidth=1, zorder=3)
        # Plot the decade value in the pointer arrow
        rounded_decade=round(2*decade)/2 # round value to 0.5 steps
        fig_scale=10**rounded_decade
        ax.text(radians, 3.3, f'10^{rounded_decade}', color='red', fontsize=12, ha='center', va='center')
       
        # Print the original scale with various conversions
        print(f"{fig_scale:.2e} | " +
      f"{convert_to_engineering_units(fig_scale)} | " +
      f"{convert_to_user_units(fig_scale)}")
        
        highlight_angle = round(angle / 30) * 30
        plot_segment(ax, highlight_angle, 30, 2.2, 0.2, 'lightgreen', 0.3)
        
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1)
        ax.set_xticks([])
        ax.set_yticks([])
        
        plt.show()

# Widgets and slider plotting

# Initialize slider for half-decade increments using FloatSlider
decade_slider = widgets.FloatSlider(
    value=0,  # Initial decade
    min=-19.0,  # Minimum value of the slider
    max=27.5,  # Maximum value of the slider
    step=0.5,  # Step size for the slider to increment in half-decade steps
    description='Decade:',  # Label of the slider
    continuous_update=True  # Update the value continuously as the slider moves
)

# Output widget to display plot
output = widgets.Output()

# Parameters for polar arrow radius
min_radius = 0.1  # Minimum radius of the arrow
max_radius = 3  # Maximum radius of the arrow

# Link the slider's value change to the update function
def on_polar_value_change(change):
    update_polar_arrow(change['new'], min_radius, max_radius)

decade_slider.observe(on_polar_value_change, names='value')

# Displaying the slider and the output plot
display(decade_slider, output)

# Call update_polar_arrow initially to display the initial state
update_polar_arrow(decade_slider.value, min_radius, max_radius)


FloatSlider(value=0.0, description='Decade:', max=27.5, min=-19.0, step=0.5)

Output()

## test: import converters

In [3]:
from convert.converters import convert_to_engineering_units, convert_to_user_units

# Example usage
print(convert_to_engineering_units(0.000001))
print(convert_to_user_units(1000))


1 micrometers (-6)
1 in kilometers


### testataan user_units....

In [6]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from convert.converters import convert_to_engineering_units, convert_to_user_units

# Initialize the slider for half-decade increments
decade_slider = widgets.FloatSlider(
    value=0,  # Initial decade
    min=-19.0,  # Minimum value of the slider
    max=27.5,  # Maximum value of the slider
    step=0.5,  # Step size for the slider to increment in half-decade steps
    description='Decade:',  # Label of the slider
    continuous_update=True  # Update the value continuously as the slider moves
)

# Output widget to display plot
output = widgets.Output()

# Define the function to update the polar plot
def update_polar_arrow(decade, min_radius, max_radius):
    radius = min_radius + (max_radius - min_radius) * ((decade + 19) / (27.5 + 19))  # Scale radius based on slider
    theta = 1  # Fixed angle for simplicity

    with output:
        output.clear_output(wait=True)
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
        ax.set_ylim(0, max_radius)  # Set the limit of the polar plot

        # Convert the distance based on the decade slider
        distance_m = 10 ** decade  # Calculate distance as 10 to the power of the slider value
        converted_distance = convert_to_user_units(distance_m)
        
        # Plot the distance as an arrow on a polar plot
        ax.arrow(theta, 0, 0, radius, alpha=0.5, width=0.015, head_width=0.05, head_length=0.1, fc='blue', ec='blue')
        ax.text(theta, radius + 0.1, converted_distance, ha='center', va='bottom')  # Display the converted distance

        plt.show()

# Link the slider's value change to the update function
decade_slider.observe(lambda change: update_polar_arrow(change['new'], min_radius, max_radius), names='value')

# Displaying the slider and the output plot
display(decade_slider, output)

# Call update_polar_arrow initially to display the initial state
update_polar_arrow(decade_slider.value, min_radius, max_radius)


FloatSlider(value=0.0, description='Decade:', max=27.5, min=-19.0, step=0.5)

Output()